# Assessment Problems Notebook

## Problem 1: Extending the Lady Tasting Tea

Let's extend the Lady Tasting Tea experiment as follows. The original experiment has 8 cups: 4 tea-first and 4 milk-first. Suppose we prepare 12 cups: 8 tea-first and 4 milk-first. A participant claims they can tell which was poured first.

Simulate this experiment using numpy by randomly shuffling the cups many times and calculating the probability of the participant correctly identifying all cups by chance. Compare your result with the original 8-cup experiment.

In your notebook, explain your simulation process clearly, report and interpret the estimated probability, and discuss whether, based on this probability, you would consider extending or relaxing the p-value threshold compared to the original design.

### Setup

To start, we import NumPy for the simulation and use `comb` from the Python
standard library to calculate the analytical probabilities. Setting a random
seed ensures that the results are reproducible each time the notebook runs.

In [5]:
import numpy as np
from math import comb

np.random.seed(35)

### Experiment setup

We define the number of total cups and how many are tea-first vs. milk-first.  
For the extended version of the experiment, we have 12 cups in total: 8 tea-first and 4 milk-first.

We will store this information in simple variables so we can use them in both the analytical calculation and the simulation later.

In [6]:
# Experiment parameters for Problem 1

n_total = 12       # total cups
n_tea_first = 8    # number of tea-first cups
n_milk_first = 4   # number of milk-first cups

# Check that our counts match the total
assert n_tea_first + n_milk_first == n_total

### Analytical probability

Before running the simulation, it helps to calculate the exact probability of getting all 12 cups correct just by guessing.  
There are 12 cups in total, and the participant needs to correctly identify which 8 are tea-first.

The number of different ways to choose 8 cups out of 12 is given by the combination “12 choose 8”.  
Since only one of these possible arrangements is completely correct, the probability of getting every cup right by chance is:

    probability = 1 / (12 choose 8)

I calculate this value below using Python.

In [7]:
# Analytical probability for the 12-cup experiment
analytical_prob_12 = 1 / comb(n_total, n_tea_first)
analytical_prob_12

0.00202020202020202